# Proyek Tugas Akhir Microcredential Associate Data Scientist

* Nama Peserta:
  - Arwendy Melyndra
  - Julius Ivander Massie
  - Samuel Goesniady
  - Maria Auleria  
* Universitas Host: UGM
* Kelas: 03
* Kelompok: 1
* Tema Project Kelompok: Deteksi Penggunaan Masker

#### Deskripsi Masalah
Hingga saat ini, pandemi covid-19 terus melanda. Terdapat banyak upaya yang telah dilakukan oleh pemerintah, salah satunya adalah Pemberlakuan Pembatasan Kegiatan Masyarakat atau dikenal dengan PPKM. Berdasarkan data yang dilansir covid19.go.id, program tersebut sukses menekan kasus covid-19 di Indonesia. Akan tetapi, masyarakat Indonesia tidak boleh lengah dan terus mematuhi protokol kesehatan, salah satunya adalah dengan mengenakan masker. Namun nyatanya, banyak masyarakat Indonesia tidak mengenakan masker ketika berada diluar rumah sehingga dikemukakan sistem deteksi pemakaian masker. Tantangan utama dalam sistem deteksi pemakaian masker ini adalah bagaimana mengenali sebuah gambar seseorang memakai masker dan tidak memakai masker yang dapat mengetahui pemakaian masker secara benar atau tidak. Sehingga permasalahan utama yang diselesaikan adalah bagaimana mengimplementasikan metode deteksi area wajah pada gambar.

**NOTE**: File notebook ini berisi gabungan dari seluruh file project yang dilakukan dan notebook ini hanya menjalankan proses pelatihan model dan evaluasi model. Pada proyek ini kami menggunakan **Anaconda** (Sebagai proses data gathering dan preparation) dan **Google Colab** (Sebagai proses data training)

***STRUKTUR PROYEK SEBELUM MELAKUKAN DATA PREPARATION***
```
Project
|   README.md
|
|___addson
|
|___dataset
|   |___images
|   |___labels
|
|___model
|   |___yolov5
|       
|___notebook
|      index.ipynb
|      1. Scraping image.ipynb
|      2. Data Preparation.ipynb
|   |___assets
```

Keterangan Project Folder:
* ***addson*** : Merupakan folder yang berisi aplikasi tambahan yang digunakan sebagai penunjang proyek. Pada proyek ini kami menggunakan labelimg untuk proses data preparation
* ***dataset***: Merupakan folder yang berisi dataset yang dikumbulkan
    * images: Berisi gambar yang diperoleh dari proses scraping dataset
    * labels: Berisi anotasi yang diperoleh dari proses labeling gambar
* ***model***: Merupakan folder yang berisi model yang akan digunakan dalam proyek. Pada proyek ini kami menggunakan yolov5
* ***notebook***: Merupakan folder yang berisi source code dari proyek
    * assets: Berisi asset penunjang notebook

## Proses Pengerjaan Proyek

### 1. Pengumpulan Dataset
Untuk mengimplementasi proyek ini, dataset yang diperlukan adalah berbentuk gambar. Untuk memperoleh data tersebut, kami melakukan scraping pada situs google image menggunakan ***BeautifulSoup***.

Dibawah ini merupakan *source code* yang kami gunakan untuk melakukan proses scraping dataset. Proses ini telah dijalankan melalui file jupyter notebook terpisah dengan nama ***1. Scraping image.ipynb***.

Setelah mengumpulkan dataset, langkah selanjutnya adalah melakukan filter secara manual, dimana kami memilah gambar-gambar agar gambar relefan dengan tujuan proyek.

Adapula file dataset yang telah kami kumpulkan melalui link berikut: [LINK DATASET](https://drive.google.com/drive/folders/152ZODHk0ddncvNqf1SHNlpw5w5tuJaIZ?usp=sharing)

In [ ]:
import requests
import os
import re
import json
import shutil
from bs4 import BeautifulSoup as bs

In [ ]:
def get_all_images_url(url, params, headers):
    """
    Fungsi ini digunakan untuk mengambil seluruh source code dari situs google image
    dan melakukan ekstraksi sehingga menghasilkan return value berupa link
    gambar yang di scraping.
    
    Parameter:
    url: string = merupakan link url yang ingin di scrape
    params: dictionary = merupakan parameter dari url
    headers: dictionary = merupakan header dari url 
    """
    urls = []
    
    html = requests.get(url, params=params, headers=headers)
    soup = bs(html.text, "html.parser")
    
    # this steps could be refactored to a more compact
    all_script_tags = soup.select('script')
    
    # https://regex101.com/r/48UZhY/4 [Hasil]
    matched_images_data = ''.join(re.findall(r"AF_initDataCallback\(([^<]+)\);", 
                                             str(all_script_tags)))
    
    # if you try to json.loads() without json.dumps it will throw an error:
    # "Expecting property name enclosed in double quotes"
    matched_images_data_fix = json.dumps(matched_images_data)
    matched_images_data_json = json.loads(matched_images_data_fix)
    
    # https://regex101.com/r/pdZOnW/3 [Hasil]
    matched_google_image_data = re.findall(r'\[\"GRID_STATE0\",null,\[\[1,\[0,\".*?\",(.*),\"All\",', 
                                           matched_images_data_json)
    
    # https://regex101.com/r/NnRg27/1 [Hasil]
    matched_google_images_thumbnails = ', '.join(
        re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]',
                   str(matched_google_image_data))).split(', ')
    
    # removing previously matched thumbnails for easier full resolution image matches.
    removed_matched_google_images_thumbnails = re.sub(
        r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', '', 
        str(matched_google_image_data))
    
    # https://regex101.com/r/fXjfb1/4
    # https://stackoverflow.com/a/19821774/15164646
    matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]",
                                                       removed_matched_google_images_thumbnails)
    
    for index, fixed_full_res_image in enumerate(matched_google_full_resolution_images):
        # https://stackoverflow.com/a/4004439/15164646 comment by Frédéric Hamidi
        original_size_img_not_fixed = bytes(fixed_full_res_image, 'ascii').decode('unicode-escape')
        original_size_img = bytes(original_size_img_not_fixed, 'ascii').decode('unicode-escape')
        urls.append(original_size_img)
    return urls

In [ ]:
def download(url, pathname):
    """
    Fungsi ini digunakan untuk mengunduh seluruh gambar
    dari url yang diperoleh dari hasil ekstraksi situs google image.
    Hasil unduhan tersebut kemudian akan disimpan kedalam path yang telah ditentukan
    
    Parameter:
    url: string = link gambar yang ingin diunduh
    pathname: string = path atau jalur folder tempat gambar diunduh
    """
    try:
        response = requests.get(url, stream=True)
        filename = re.sub(r'[^ \nA-Za-z0-9/]+', '', url)
        filepath = os.path.join(pathname, filename.split("/")[-1])
    
        file = open("{}.jpg".format(filepath), "wb")
        response.raw.decode_content = True
        shutil.copyfileobj(response.raw, file)
        print("{} downloaded".format(filename))
    except:
        pass
    

In [ ]:
def main(url, params, path):
    """
    Fungsi ini berisi persiapan untuk menjalankan
    fungsi get_all_images_url, dan download
    
    Parameter:
    url: string = merupakan link url yang ingin di scrape
    params: dictionary = merupakan parameter dari url
    path: string = path atau jalur folder tempat gambar diunduh
    """
    # Persiapan URL
    headers = {
       "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }
    img_urls = get_all_images_url(url, params, headers)
    for url in img_urls:
        download(url, path)

In [ ]:
params = {
    "q": "orang jogging gbk", #Search nya apa
    "tbm": "isch",
    "ijn": "0"
}
url = "https://www.google.com/search"
path = "..\dataset"

main(url, params, path)

### 2. Eksplorasi dan Persiapan Data
Setelah melakukan proses pengumpulan data, langkah selanjutnya adalah dengan melakukan eksplorasi data dan melakukan persiapan dataset.

Proses persiapan dataset meliputi beberapa proses:

1. ***Pelabelan dataset*** <br>
Sebelum melakukan pelabelan dataset, kami menerapkan model YOLOv5 untuk mengimplementasi proyek. Berdasarkan [sumber](https://www.section.io/engineering-education/introduction-to-yolo-algorithm-for-object-detection/), YOLO merupakan salah satu algoritma yang digunakan untuk menerapkan object detection. Dalam prosesnya, agar model dapat mengindentifikasi dataset, kita perlu melakukan pelabelan terhadap objek dari masing-masing gambar dan disimpan dalam bentuk file txt yang berisi anotasi dari objek serta label dari objek. <br><br>
Untuk mengimplementasi hal tersebut, kami menggunakan [labelimg](https://github.com/tzutalin/labelImg) untuk melakukan pelabelan data. LabelImg dapat dilakukan instalasi dengan menggunakan perintah pada console berikut:<br><br>
    ```git clone https://github.com/tzutalin/labelImg.git ```<br><br>
Instalasi tersebut berbentuk folder proyek dan agar LabelImg dapat berjalan, arahkan console menuju folder proyek tersebut dan jalankan perintah berikut: <br><br>
    ```conda install pyqt=5```<br>
    ```conda install -c anaconda lxml``` <br>
    ```pyrcc5 -o libs/resources.py resources.qrc``` <br><br>
Setelah menjalankan perintah diatas, langkah selanjutnya adalah melakukan pendefinisian label dataset yakni *mask* dan *no-mask* pada file yang terletak dalam folder proyek LabelImg yakni ```data/predefined_classes.txt```. Sehingga struktur file dari ```predefined_classes.txt``` menjadi seperti berikut:
```
mask
no-mask
```
Kemudian kembali ke dalam console dengan arah direktori merupakan folder proyek LabelImg, jalankan perintah berikut: <br><br>
    ```python labelImg.py```<br><br>
Maka program labelImg akan muncul seperti tampilan [berikut](assets/labelimg.png). Langkah berikutnya adalah menekan tombol open dir untuk membuka lokasi dataset disimpan, kemudian diikuti dengan menekan tombol change save dir sebagai tempat menyimpan lokasi label, langkah ini kemudian diakhiri dengan mengubah format Pascal VOC menjadi YOLO dengan menekan tombol Pascal VOC hingga menemukan YOLO. Maka dari itu gambar dapat dilabel sesuai dengan label yang diberikan dan disimpan dengan menekan tombol save. Setelah melakukan proses save, program akan menyimpan anotasi dari label berbentuk txt dan menyimpan jenis label dengan nama file *classess.txt*. Setelah selesai melakukan pelabelan, langkah terakhir adalah memindahkan file *classess.txt* keluar dari direktori save file label<br><br>
Pada kasus ini, kami telah menyediakan dataset yang sudah dilabel melalui link berikut: [LINK DATASET](https://drive.google.com/drive/folders/1HT0zIiiu9arMlLFgpCXqi7_7ZiYxMF3a?usp=sharing)

2. ***Eksplorasi dan Persiapan dataset*** <br>
Setelah melakukan pelabelan dataset, langkah selanjutnya adalah melakukan eksplorasi dataset yang meliputi menghitung jumlah label, jumlah gambar, serta menvisualisasikan gambar beserta label yang telah dilabelkan. Persiapan dataset dilakukan dengan pemisahan dataset, serta pembuatan YAML sebagai penunjang model. Proses ini telah dijalankan melalui file jupyter notebook terpisah dengan nama ***2. Data Preparation.ipynb***.<br><br>
Setelah melakukan proses data preparation, langkah selanjutnya adalah dengan membuat file YAML yang kemudian disimpan ke dalam folder ```workspace\models\v1``` dan  dengan nama ```custom_dataset.yaml``` yang berisi data berikut:<br>
```
path: ../../dataset/
train: images/train
val: images/test
nc: 2  
names: [ 'mask', 'no-mask' ]
```

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
import cv2
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# DIREKTORI FILE DATASET DAN ANOTASINYA
ANNOTATION_DIR = "../dataset/labels/"
IMAGE_DIR = "../dataset/images/"

In [ ]:
# MENYIMPAN INFORMASI SEMUA ANOTASINYA KE DALAM DATAFRAME
datas = {
    "nama_file": [],
    "class": [],
    "center_x": [],
    "center_y": [],
    "width": [],
    "height": []
}

for file in os.listdir(ANNOTATION_DIR):
    if file.endswith("txt"):
        file_path = "{}\{}".format(ANNOTATION_DIR, file)
        with open(file_path, "r") as f:
            for line in f.readlines():
                row = line.split()
                datas["nama_file"].append(file.split(".")[0])
                datas["class"].append(row[0])
                datas["center_x"].append(row[1])
                datas["center_y"].append(row[2])
                datas["width"].append(row[3])
                datas["height"].append(row[4])

df = pd.DataFrame(datas)
df["class"].replace({"0": "mask", "1": "no_mask"}, inplace=True)

In [ ]:
df

In [ ]:
jumlah_gambar = df["nama_file"].nunique()
print(f"Jumlah Gambar Dalam Dataset: {jumlah_gambar}")

In [ ]:
label_per_gambar = df["nama_file"].value_counts().to_frame().reset_index(drop=False)
label_per_gambar.rename(columns={"index": "nama_file", "nama_file":"jumlah_label"}, inplace=True)
label_per_gambar

In [ ]:
df.info()

In [ ]:
# Mengubah tipe data class, center_x, center_y, width, height jadi float
df = df.astype({"center_x": float, "center_y": float, "width": float, "height": float})
df.info()

In [ ]:
jumlah_kelas = df["class"].value_counts().to_frame().reset_index(drop=False)
jumlah_kelas.rename(columns={"index": "class", "class":"jumlah_data"}, inplace=True)
jumlah_kelas

In [ ]:
def get_target_ds(name, df):
    """
    Fungsi ini digunakan untuk mendapatkan anotasi dari label yang
    diperoleh dari csv yang berisi informasi mengenai label dan gambar dataset
    """
    bboxes_cols = ["center_x", "center_y", "width", "height"]
    rows = df[df["nama_file"] == name[:-4]]
    return rows["class"].values, rows[bboxes_cols].values


def get_bb(bboxes, img):
    """
    Fungsi ini digunakan untuk mendapatkan lebar dan tinggi label, serta posisi dari label
    Hal ini diperlukan karena format labeling YOLO sebagai berikut:
    [x_center, y_center, width, height]
    
    x_center: posisi x-axis label yang dinormalisasi
    y_center posisi y-axis label yang dinormalisasi
    width: lebar label yang dinormalisasi
    height: tinggi label yang dinormalisasi
    """
    boxes = bboxes.copy()
    boxes[:,0] = (boxes[:,0] - boxes[:,2]*0.5)*img.shape[1]
    boxes[:,1] = (boxes[:,1] - boxes[:,3]*0.5)*img.shape[0]
    boxes[:,2] = boxes[:,2] * img.shape[1]
    boxes[:,3] = boxes[:,3] * img.shape[0]
    
    if boxes.shape[0] == 1 : return boxes
    return np.squeeze(boxes)


def img_show(img, ax = None, figsize=(7,11)):
    """ 
    Fungsi ini digunakan untuk menvisualisasikan gambar
    """
    if ax is None: fig, ax = plt.subplots(figsize=figsize)
    ax.xaxis.tick_top()
    ax.imshow(img)
    ax.axis("off")
    
    return ax


def draw_outline(obj):
    """
    Fungsi ini digunakan untuk menambahkan outline pada box / text
    """
    obj.set_path_effects([patheffects.Stroke(linewidth=4, foreground="black"), patheffects.Normal()])


def draw_box(img, ax, bb, color):
    """
    Fungsi ini digunakan untuk menggambarkan label yang akan divisualisasikan
    """
    patch = ax.add_patch(patches.Rectangle((bb[0], bb[1]), bb[2], bb[3], fill=False, edgecolor=color, lw=2))
    draw_outline(patch)

def draw_text(ax, bb, txt, disp, color):
    """
    Fungsi ini digunakan untuk menggambarkan class dari label yang akan divisualisasikan
    """
    text = ax.text(bb[0], bb[1]-disp, txt, 
                   verticalalignment="bottom", 
                   color=color, 
                   fontsize=24, 
                   )

def plot_sample(img, bboxes, labels, ax=None, figsize=(18,10)):
    """
    Fungsi ini untuk menampilkan 1 buah gambar beserta dengan labelnya
    """
    bb = get_bb(bboxes, img)
    ax = img_show(img, ax=ax)
    for i in range(len(bboxes)):
        if labels[i] == "mask":
            color = "red"
        elif labels[i] == "no_mask":
            color = "green"
        draw_box(img, ax, bb[i], color)
        draw_text(ax, bb[i], str(labels[i]), img.shape[0]*0.05, color)
        
def multiplot(dim: tuple, df, images, idxs=None, figsize=(18,10)):
    """
    Fungsi ini untuk menampilkan lebih dari satu gambar beserta dengan labelnya
    """
    if idxs is None: idxs = np.random.randint(0, len(images)-1, dim[0]*dim[1])
    fig, ax = plt.subplots(dim[0], dim[1], figsize=figsize)
    plt.subplots_adjust(wspace=0.1, hspace=0)
    fig.tight_layout()
    length = dim[0]*dim[1]
    for i in range(dim[0]):
        for j in range(dim[1]):
            length-=1
            img = images[idxs[length]]
            img_path = IMAGE_DIR + img
            labels, bboxes = get_target_ds(img, df)
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            plot_sample(img, bboxes, labels, ax=ax[i][j])

In [ ]:
images = os.listdir(IMAGE_DIR)
multiplot((3,3), df, images, figsize=(17,17))

In [ ]:
plt.figure(figsize=(7,4))
sns.barplot(x="class", y="jumlah_data", data=jumlah_kelas)
plt.title("Jumlah Label Dataset")
plt.show()

In [ ]:
def move_file(list_file, destination_path):
    """
    Fungsi ini digunakan untuk memindahkan file image dan anotasi menuju file yang diinginkan
    """
    if not os.path.exists(destination_path[0]):
        os.mkdir(destination_path[0])
    if not os.path.exists(destination_path[1]):
        os.mkdir(destination_path[1])
        
    for file in list_file:
        image = file+".jpg"
        txt = file+".txt"
        shutil.move(os.path.join(IMAGE_DIR, image),
                    os.path.join(destination_path[0], image))
        
        shutil.move(os.path.join(ANNOTATION_DIR, txt),
                    os.path.join(destination_path[1], txt))
        
        
# Split dataset
file_name = label_per_gambar["nama_file"]
train_image, test_image = train_test_split(file_name, test_size=0.1)

train_path = [IMAGE_DIR+"train/", ANNOTATION_DIR+"train/"]
test_path = [IMAGE_DIR+"test/", ANNOTATION_DIR+"test/"]

move_file(train_image, train_path)
move_file(test_image, test_path)

### 3. Pelatihan Model
Setelah melakukan berbagai macam preparasi, langkah selanjutnya adalah melakukan pelatihan model. Pada proyek ini, kami menggunakan YOLOv5 yang dapat diperoleh dengan menjalankan perintah pada console sebagai berikut: <br>

```
git clone https://github.com/ultralytics/yolov5
```

Setelah mengunduh model, langkah selanjutnya adalah melakukan instalasi depedensi yang terdapat dalam model. Kemudian langkah berikutnya adalah dengan menjalankan proses training dan diakhiri dengan proses evaluasi seperti pada source code dibawah.

Pada proses ini, kami menjalankan program tersebut menggunakan google colab dan mengaktifkan GPU mode pada google collab.

In [ ]:
# Pada proyek ini, kami menyimpan project ke dalam google drive, sehingga diperlukan syntax berikut
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Berdasarkan tempat project disimpan
%cd '/content/gdrive/My Drive/Tugas-Akhir-MCDS/Project/model/yolov5/'
%pip install -qr requirements.txt

/content/gdrive/My Drive/Tugas-Akhir-MCDS/Project/model/yolov5
     |████████████████████████████████| 596 kB 10.6 MB/s 


Setelah melakukan training, hasil program disimpan ke dalam folder ```model/yolov5/runs/train/custom_model3```.

File tersebut berisi:
1. Weight dari custom model (baik yang terakhir dan terbaik)
2. Hasil validasi model (baik yang diprediksi dengan yang aslinya)
3. Hasil training model (baik yang diprediksi dengan yang aslinya)
4. Grafik yang berisi loss, akurasi, confussion matrix, dll

Jika kita ketahui, hasil yang diperoleh adalah sebanyak 3 folder, dimana 2 folder lainnya yakni ```custom_model``` dan ```custom_model1``` merupakan hasil pelatihan model dari epoch sebelumnya. Pembagian tersebut menurut kami adalah sebuah checkpoint dari model ketika melakukan proses training.

In [ ]:
!python train.py --img 640 --batch 8 --epochs 75 --data data/custom_dataset.yaml --weights yolov5s.pt --name custom_model

train: weights=yolov5s.pt, cfg=, data=data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=75, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=custom_model, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2021-12-6 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

In [ ]:
# Melakukan testing dengan data validasi
!python detect.py --weights runs/train/custom_model3/weights/best.pt --img 640 --conf 0.5 --source ../../dataset/validation/

detect: weights=['runs/train/custom_model3/weights/best.pt'], source=../../dataset/validation/, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-124-g1075488 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/2 (1/376) /content/gdrive/My Drive/Tugas-Akhir-MCDS/Project/dataset/validation/video-1.mp4: 384x640 5 masks, Done. (0.029s)
video 1/2 (2/376) /content/gdrive/My Drive/Tugas-Akhir-MCDS/Project/dataset/validation/video-1.mp4: 384x640 5 masks, Done. (0.027s)
video 1/2 (3/376) /content/gdrive/My Drive/Tugas-Akhir-MCDS/Project/dataset/validation/video-1.mp4: 384x640 4 masks, D

In [ ]:
# Export Model
!python export.py --weights runs/train/custom_model3/weights/best.pt --img 640 --include tfjs

export: data=data/coco128.yaml, weights=['runs/train/custom_model3/weights/best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=14, verbose=False, workspace=4, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tfjs']
YOLOv5 🚀 2021-12-6 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from runs/train/custom_model3/weights/best.pt (14.5 MB)

TensorFlow saved_model: starting export with tensorflow 2.7.0...

                 from  n    params  module                                  arguments                     
2021-12-12 03:37:28.383776: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1